In [5]:
import pandas as pd
import numpy as np
data = pd.read_csv('diabetes1.csv')

from sklearn.model_selection import train_test_split
X = np.array(data.iloc[:,0:-1])
y =  np.array(data.iloc[:,-1])

X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.30,random_state =1)

from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train,y_train)
ypred = model.predict(X_test)
print("Data avaialable :",len(data))
print("Training Data :",len(X_train))
print("Testing Data :",len(X_test))
print("Instance\tTest\tPrediction")
for i in range(len(X_test)):

        print(i+1,"\t\t",y_test[i],"\t",ypred[i])
    
from sklearn import metrics
print("Accuracy :",str(round(metrics.accuracy_score(y_test,ypred)*100,2)),"%")

Data avaialable : 767
Training Data : 536
Testing Data : 231
Instance	Test	Prediction
1 		 0 	 1
2 		 0 	 0
3 		 0 	 1
4 		 0 	 0
5 		 0 	 0
6 		 0 	 0
7 		 0 	 0
8 		 1 	 0
9 		 0 	 0
10 		 0 	 0
11 		 0 	 1
12 		 0 	 0
13 		 0 	 0
14 		 0 	 0
15 		 0 	 0
16 		 1 	 0
17 		 0 	 1
18 		 0 	 0
19 		 0 	 0
20 		 0 	 0
21 		 0 	 0
22 		 1 	 0
23 		 0 	 0
24 		 1 	 0
25 		 0 	 0
26 		 0 	 0
27 		 0 	 1
28 		 0 	 0
29 		 0 	 0
30 		 1 	 1
31 		 0 	 0
32 		 0 	 0
33 		 0 	 0
34 		 1 	 1
35 		 0 	 0
36 		 1 	 1
37 		 0 	 0
38 		 0 	 1
39 		 1 	 1
40 		 0 	 0
41 		 1 	 0
42 		 0 	 0
43 		 1 	 0
44 		 1 	 0
45 		 0 	 0
46 		 0 	 0
47 		 1 	 0
48 		 1 	 0
49 		 0 	 0
50 		 0 	 0
51 		 0 	 0
52 		 0 	 0
53 		 0 	 0
54 		 0 	 1
55 		 0 	 0
56 		 0 	 0
57 		 0 	 0
58 		 1 	 1
59 		 0 	 0
60 		 0 	 0
61 		 1 	 1
62 		 1 	 0
63 		 0 	 0
64 		 1 	 0
65 		 1 	 0
66 		 0 	 0
67 		 0 	 0
68 		 1 	 0
69 		 0 	 0
70 		 0 	 1
71 		 0 	 1
72 		 0 	 1
73 		 0 	 0
74 		 0 	 0
75 		 1 	 1
76 		 1 	 1
77 		 0 	 1